# Step-1
### Import the Package 

In [1]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import nltk

nltk.download('punkt')
nltk.download('stopwords')

C:\Users\Shree\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\Shree\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\Shree\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "class": algorithms.TripleDES,
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Shree\AppData\Roaming

True

# Step-2
### Read the Data 

In [2]:
twitter_df = pd.read_csv(r"C:\Users\Shree\OneDrive\Desktop\FSDS_omkar sir\Datafiles\twitter_training.csv")  # Load uploaded dataset
twitter_df.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


### Drop '2401' and Borderlands Columns

In [3]:
df = twitter_df.drop(columns=['2401','Borderlands'])
df

,Positive,"im getting on borderlands and i will murder you all ,"
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...
...,...,...
74676,Positive,Just realized that the Windows partition of my...
74677,Positive,Just realized that my Mac window partition is ...
74678,Positive,Just realized the windows partition of my Mac ...
74679,Positive,Just realized between the windows partition of...


In [4]:
TS_df = df.rename(columns={'Positive': 'Sentiment','im getting on borderlands and i will murder you all ,':'Review'})
TS_df

,Sentiment,Review
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...
...,...,...
74676,Positive,Just realized that the Windows partition of my...
74677,Positive,Just realized that my Mac window partition is ...
74678,Positive,Just realized the windows partition of my Mac ...
74679,Positive,Just realized between the windows partition of...


In [5]:
TS_df['Review'][0]

'I am coming to the borders and I will kill you all,'

# Step-3
### Preprocess the text data

In [6]:
def preprocess(text):
    if not isinstance(text, str):  # Check if input is not a string , If text is not a string then return Empty String("")
        return ""

    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
    text = text.lower()
    text = text.strip()
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = stopwords.words("english")
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens


# Apply preprocessing to the text data
TS_df['tokens'] = TS_df['Review'].apply(preprocess)
TS_df

,Sentiment,Review,tokens
0,Positive,I am coming to the borders and I will kill you...,"[coming, borders, kill]"
1,Positive,im getting on borderlands and i will kill you ...,"[im, getting, borderlands, kill]"
2,Positive,im coming on borderlands and i will murder you...,"[im, coming, borderlands, murder]"
3,Positive,im getting on borderlands 2 and i will murder ...,"[im, getting, borderlands, murder]"
4,Positive,im getting into borderlands and i can murder y...,"[im, getting, borderlands, murder]"
...,...,...,...
74676,Positive,Just realized that the Windows partition of my...,"[realized, windows, partition, mac, like, year..."
74677,Positive,Just realized that my Mac window partition is ...,"[realized, mac, window, partition, years, behi..."
74678,Positive,Just realized the windows partition of my Mac ...,"[realized, windows, partition, mac, years, beh..."
74679,Positive,Just realized between the windows partition of...,"[realized, windows, partition, mac, like, year..."


# Step-4
### Train the Word2Vec Model

In [7]:
model = Word2Vec(sentences=TS_df['tokens'], vector_size=100, window=5, min_count=1, sg=1)

In [8]:
model

**Model Vocabulary**

In [9]:
vocabulary_list = list(model.wv.index_to_key)
print(len(vocabulary_list))

38222


**Word Vector**

In [10]:
TS_df['tokens'][0]

['coming', 'borders', 'kill']

In [11]:
len(model.wv['coming'])

100

# Step-5
### Create Sentence Vectors

In [12]:
def get_sentence_vector(tokens, model):
    word_vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(100)  # Assuming word vectors have size 100
    return np.mean(word_vectors, axis=0)

# Apply function to create sentence vectors for TS_df
TS_df['sentence_vector'] = TS_df['tokens'].apply(lambda tokens: get_sentence_vector(tokens, model))

# Convert to NumPy arrays
X = np.vstack(TS_df['sentence_vector'])
y = TS_df['Sentiment'] # Labels 

In [13]:
print(X.shape,y.shape)

(74681, 100) (74681,)


# Step-6
### Train Test Classifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Train the Classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

In [ ]:
### Evalute the Model